In [ ]:
#AI Trainer for the Adjustments Project


In [26]:
#Load input data
import pandas as pd
import numpy as np

#Load data
df=pd.read_csv('SampleData.csv')
df.head(10)

,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,GRC_EndMonth,RiskCategory,727232,748121,0.12,CUST450896,SME,Medium,1,GB,Medium
1,Amend,Fix_Broken_Batch,CustomerType,617646,396342,0.71,CUST359147,Retail,Low,1,GB,Retail
2,Amend,Fix_Broken_Batch,CountryCode,25383,980338,0.49,CUST563201,Corporate,Medium,0,US,DE
3,Amend,Fix_Broken_Batch,RiskCategory,217640,28801,0.54,CUST981372,Corporate,Low,0,DE,Low
4,Amend,GRC_EndMonth,IsDefaulted,950796,801071,0.50,CUST665428,SME,Medium,1,DE,0
5,Amend,GRC_EndMonth,CustomerID,734216,366855,0.00,CUST712035,SME,High,1,GB,CUST238947
6,Amend,GRC_EndMonth,RiskCategory,589877,812864,0.58,CUST238947,Retail,Low,0,DE,High
7,Amend,GRC_EndMonth,CustomerID,527559,746509,0.20,CUST192654,Retail,Low,0,GB,CUST450896
8,Amend,Defaulting_Reclassify,RiskCategory,834443,986900,0.92,CUST450896,Retail,Low,0,GB,High
9,Amend,Default,IsDefaulted,566399,640410,0.10,CUST238947,Retail,Low,0,DE,0


In [27]:
#Stage 1: OneHot encode string labels so we can use them in the model

#Build list of all string columns in the dataframe
string_columns=[]
for col in df.columns:
    if df[col].dtype=='object':
        string_columns.append(col)


df_encoded=pd.get_dummies(df, columns=string_columns)
df_encoded


,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,NewVal_CUST981372,NewVal_Corporate,NewVal_DE,NewVal_GB,NewVal_High,NewVal_Low,NewVal_Medium,NewVal_Retail,NewVal_SME,NewVal_US
0,727232,748121,0.12,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,617646,396342,0.71,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,25383,980338,0.49,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,217640,28801,0.54,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,950796,801071,0.50,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,318438,473192,0.94,1,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
996,786830,25495,0.61,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
997,71999,152908,0.27,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
998,16019,509389,0.30,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [32]:
#Stage 2: Normalize all numeric values
#Normalize the dataframe
# df_normalized = (df_encoded - df_encoded.mean()) / df_encoded.std()

# #No idea why this one breaks
# #"AdjustmentType_Amend"
# df_normalized["AdjustmentType_Amend"]=1

#Normalize each column, one at a time - except the onehot encoded ones
df_normalized=df_encoded.copy()
for col in df_encoded.columns:
    #Get everything in column before _
    col_name=col.split('_')[0]
    if col_name not in string_columns:
        df_normalized[col]=(df_encoded[col]-df_encoded[col].mean())/df_encoded[col].std()



# df_normalized=df_encoded.copy()
# for col in df_encoded.columns:
#     df_normalized[col]=(df_encoded[col]-df_encoded[col].mean())/df_encoded[col].std()

df_normalized

,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,NewVal_CUST981372,NewVal_Corporate,NewVal_DE,NewVal_GB,NewVal_High,NewVal_Low,NewVal_Medium,NewVal_Retail,NewVal_SME,NewVal_US
0,0.866225,0.861521,-1.184078,1.036154,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,0.474344,-0.334582,0.851072,1.036154,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,-1.643596,1.651095,0.092203,-0.964143,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,-0.956082,-1.584279,0.264673,-0.964143,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1.665693,1.041559,0.126697,1.036154,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.595627,-0.073280,1.644436,1.036154,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
996,1.079349,-1.595520,0.506131,-0.964143,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
997,-1.476896,-1.162296,-0.666667,1.036154,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
998,-1.677082,0.049795,-0.563185,1.036154,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [31]:
#Split the data into test and training sets using pandas
msk = np.random.rand(len(df_normalized)) < 0.8
train = df_normalized[msk]
test = df_normalized[~msk]


In [35]:
#FUN time - lets build a neural network!
import torch
import torch.nn as nn
import torch.optim as optim

#Prep work 
#Get number of onehot columns created from NewVal
outputs=len(df["NewVal"].unique())

# number of features (len of X cols)
input_dim = train.columns.shape[0]
# number of hidden layers
hidden_layers = 5
# number of classes (unique of y)
output_dim = outputs
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
  def forward(self, x):
    x = torch.sigmoid(self.linear1(x))
    x = self.linear2(x)
    return x
